This is the Coursera Capstone for IBM Professional Certification 

In [18]:
import pandas as pd
import numpy as np

In [19]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Web Scrapping to get the Toronto's Postal Code. 
### 3rd Week Assignment Part 1.

In [84]:
#import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# we import the libraries for request and BeautifulSoup
from urllib.request import urlopen
from bs4 import BeautifulSoup


### Define the url variable to store the desired wikipedia page.

In [85]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

### Let's use the pandas' read_html method to get the table from the url given. 

In [86]:
dflst = pd.read_html(url,header =0)
df1 = dflst[0]
dflst[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore cells with a borough that is Not assigned

In [87]:
df = df1[df1.Borough != 'Not assigned']

In [88]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Let's check the Bourough that has 'Not assigned' Neighbourhood

In [89]:
df[df.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


As it is only a Row we can change the value manually

In [90]:
df.loc[8,'Neighbourhood'] = "Queen's Park"
df[df.Neighbourhood == 'Not assigned']

C:\Users\d797150\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\d797150\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood


### Let's explore the duplicated row with .duplicated method.

In [91]:
df[df.duplicated(['Postcode'])].index

Int64Index([  5,   7,  12,  16,  18,  23,  24,  25,  26,  28,
            ...
            268, 269, 270, 271, 272, 273, 283, 284, 285, 286],
           dtype='int64', length=108)

### For loop over a dataframe of the duplicated rows to get the index and Neighbourhood.
In this For loop we get the index and the Neighbourhood string into temporal variables that will be used to locate and update 
the original dataframe df.

In [92]:
#Store the duplicate DF 
dfD = df[df.duplicated(['Postcode'])]

for idx, neigh in zip(dfD.index,dfD['Neighbourhood']):
    temp = df.loc[idx-1,'Neighbourhood']
    #print(temp)
    df.loc[idx-1,"Neighbourhood"] = temp+','+neigh

C:\Users\d797150\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [93]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,"Lawrence Heights,Lawrence Manor"


### Let's get rid of the duplicated rows with drop_duplicates method.


In [94]:
df.drop_duplicates(['Postcode'], keep='first', inplace=True)
df.head(10)

C:\Users\d797150\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson,Garden District"


In [95]:
df.shape

(103, 3)

### We use pandas read_csv method to get the postcode, Lat and Long from the .csv file

In [96]:
latlon = pd.read_csv('Geospatial_Coordinates.csv')
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Let's rename the Postal Code Column with Postcode for consistency purposes

In [97]:
latlon.rename(columns={"Postal Code": "Postcode"}, inplace = True)
latlon.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### For loop over the latlon dataframe
We get the postcode from latlon and search for that postcode in the df dataframe, then the index of df is obtained. 
With the index we can update df to add the latitude and longitud information from the latlon dataframe.

In [98]:
for idx, pcode,lat,lon in zip(latlon.index, latlon['Postcode'], latlon['Latitude'], latlon['Longitude']):
    # Getting the index values of df that match the postcod of latlon dataframe. 
    i = df[df['Postcode'] == pcode].index.values
    #print(i.shape[0])
    if i.shape[0] != 0:
        df.loc[i,'Latitude'] = lat
        df.loc[i,'Longitud'] =lon
        

In [99]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitud
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
6,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
8,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [100]:
df.tail()

,Postcode,Borough,Neighbourhood,Latitude,Longitud
255,M8X,Etobicoke,"The Kingsway,Montgomery Road",43.653654,-79.506944
262,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
265,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
266,M8Y,Etobicoke,"Humber Bay,King's Mill Park",43.636258,-79.498509
282,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW",43.628841,-79.520999
